# BASELINE Framework - S2T & dpmChecks Examples

**Version:** 2.0  
**Release Date:** January 15, 2026  
**Organization:** Intel Corporation - Test & Validation  
**Classification:** Intel Confidential  
**Maintainer:** Gabriel Espinoza (gabriel.espinoza.ballestero@intel.com)  
**Products:** GNR (Granite Rapids), CWF (Clearwater Forest), DMR (Diamond Rapids)  
**Repository:** `c:\Git\Automation\S2T\`

---

This notebook demonstrates how to use the **S2T module and dpmChecks** functions across GNR, CWF, and DMR products. All functions work identically across products with product-specific import paths.

**Topics Covered:**
- MCA logging and error reporting (Logger)
- Pseudo boot script configuration changes (pseudo_bs)
- Unit information and validation functions
- BIOS knob verification
- Fuse and SKU checking
- Voltage sweep automation

**Complete Documentation:**
- User Manual: [THR_DEBUG_FRAMEWORK_USER_MANUAL.md](THR_DEBUG_FRAMEWORK_USER_MANUAL.md)
- File Naming & Imports: [THR_DEBUG_FRAMEWORK_FILE_NAMING_AND_IMPORTS.md](THR_DEBUG_FRAMEWORK_FILE_NAMING_AND_IMPORTS.md)
- DebugFramework Examples: [THR_DEBUG_FRAMEWORK_EXAMPLES.ipynb](THR_DEBUG_FRAMEWORK_EXAMPLES.ipynb)

## Setup - Import Modules

In [ ]:
# Choose ONE based on your product:

# For GNR
import users.THR.PythonScripts.thr.S2T.dpmChecksGNR as dpm

# For CWF (uncomment if using CWF)
# import users.THR.PythonScripts.thr.S2T.dpmChecksCWF as dpm

# For DMR/Next Gen (uncomment if using DMR)
# import users.THR.dmr_debug_utilities.S2T.dpmChecks as dpm

print("✓ dpmChecks module imported successfully")

---
## 1. Logger - MCA Logging and Error Reporting

The logger function captures MCA errors, generates reports, and logs test data.

### Logger - Function Signature

```python
def logger(
    visual='',              # Visual ID of unit
    qdf='',                 # QDF number
    TestName='',            # Name of test
    Testnumber=0,           # Test number
    dr_dump=True,           # Dump DR (Debug Register)
    chkmem=0,               # Check memory
    debug_mca=0,            # Debug MCA errors
    folder=None,            # Log folder path
    WW='',                  # Work week
    Bucket='UNCORE',        # Test bucket category
    UI=False,               # Use UI interface
    refresh=False,          # Refresh SV
    logging=None,           # Logging object
    upload_to_disk=False,   # Upload logs to disk
    upload_to_danta=False   # Upload to DANTA database
)
```

### Logger - Basic Usage

### Logger - GUI vs Background Mode

The logger function can run in two modes:
- **Background Mode (UI=False)**: Runs silently in the background, logging automatically
- **GUI Mode (UI=True)**: Opens interactive interface for manual logging control

### Logger - What It Does

The logger:
1. **Captures MCA Errors** - Monitors and logs Machine Check Architecture errors
2. **Decodes MCA Banks** - Interprets MCA errors for CHA, LLC, UBOX, PM, PUNIT
3. **Generates Reports** - Creates comprehensive error reports
4. **Logs Test Data** - Records test parameters and results
5. **PPV Format** - Outputs in PPV Bucketer compatible format

**MCA Banks Decoded:**
- CHA (Caching and Home Agent)
- LLC (Last Level Cache)
- UBOX (Uncore Box)
- PM (Power Management)
- PUNIT (Power Unit)

**Report Location:** `C:\SystemDebug\Logs\MCA_Reports\`

### Logger - GUI Mode Example

In [ ]:
# GUI Mode - Opens interactive logging interface
# User can control logging manually through the interface
dpm.logger(
    visual='74GC556700043',
    TestName='InteractiveTest',
    Testnumber=1,
    UI=True,                     # Opens GUI interface
    dr_dump=True,
    upload_to_disk=True
)

# The Logger GUI provides:
# - Real-time MCA error display
# - Manual log capture buttons
# - Test parameter configuration
# - Report generation controls
# - Status indicators

print("✓ Logger GUI opened - use interface to control logging")

### Logger - Background Mode Examples

Background mode (UI=False) runs automatically without user interaction.

In [ ]:
# Background Mode - Example 1: Basic automatic logging
dpm.logger(
    visual='74GC556700043',     # Your unit's Visual ID
    TestName='MeshDefeature',
    Testnumber=1,
    dr_dump=True,
    Bucket='UNCORE',
    UI=False                    # Background mode - no GUI
)

print("✓ Logger initialized in background - will capture MCA errors during test")
print("  Reports saved to: C:\\SystemDebug\\Logs\\MCA_Reports\\")

In [ ]:
# Background Mode - Example 2: Full configuration with QDF
dpm.logger(
    visual='74GC556700043',
    qdf='Q12345',                # QDF number
    TestName='VoltageMargin',
    Testnumber=5,
    dr_dump=True,                # Dump DR registers
    debug_mca=1,                 # Enable MCA debugging
    WW='2026WW03',               # Work week
    Bucket='CORE',               # Core bucket
    UI=False                     # Background mode
)

print("✓ Logger with full configuration running in background")

In [ ]:
# Background Mode - Example 3: Automated burn-in logging
dpm.logger(
    visual='74GC556700043',
    TestName='BurnIn_24hr',
    Testnumber=1,
    dr_dump=True,
    debug_mca=1,
    Bucket='FULL_CHIP',
    UI=False,                    # Background - no user interaction needed
    upload_to_disk=True,
    upload_to_danta=False
)

print("✓ 24-hour burn-in logging running in background - no GUI")

---
## 2. Pseudo Bootscript (pseudo_bs)

Pseudo bootscript allows bootscript-like configuration without actual platform reboot.

### pseudo_bs - Function Signature

```python
def pseudo_bs(
    ClassMask='FirstPass',      # Class mask type
    Custom=[],                  # Custom mask array
    boot=True,                  # Execute bootscript
    use_core=False,             # Use core mode
    htdis=True,                 # Disable HyperThreading
    dis_2CPM=None,              # Disable 2 Cores Per Module
    dis_1CPM=None,              # Disable 1 Core Per Module
    fuse_read=True,             # Read fuses
    s2t=False,                  # S2T mode
    masks=None,                 # Mask configuration
    clusterCheck=None,          # Cluster check
    lsb=False,                  # LSB mode
    fuse_compute=None,          # Fuse compute setting
    fuse_io=None,               # Fuse IO setting
    fast=False,                 # Fast mode
    ppvcfuse=False,             # PPVC fuse mode
    skip_init=False,            # Skip initialization
    vbump={                     # Voltage bump configuration
        'enabled': False,
        'type': ['cfc'],
        'offset': 0,
        'computes': ['compute0', 'compute1', 'compute2']
    }
)
```

### pseudo_bs - Basic Usage

In [ ]:
# Example 1: Simple pseudo bootscript with FirstPass mask
dpm.pseudo_bs(
    ClassMask='FirstPass',      # Use FirstPass mask
    boot=True,                  # Execute boot
    htdis=True,                 # Disable HT
    fuse_read=True              # Read fuses
)

print("✓ Pseudo bootscript executed with FirstPass mask")

In [ ]:
# Example 2: Pseudo bootscript with CPM configuration
dpm.pseudo_bs(
    ClassMask='FirstPass',
    boot=True,
    htdis=True,
    dis_1CPM=True,              # Disable 1 Core Per Module
    dis_2CPM=False,             # Keep 2 CPM enabled
    fuse_read=True
)

print("✓ Pseudo bootscript with CPM settings executed")

In [ ]:
# Example 3: Pseudo bootscript with voltage bump
dpm.pseudo_bs(
    ClassMask='FirstPass',
    boot=True,
    htdis=True,
    vbump={
        'enabled': True,
        'type': ['cfc'],        # Bump CFC voltage
        'offset': 0.05,         # +50mV
        'computes': ['compute0', 'compute1', 'compute2']
    }
)

print("✓ Pseudo bootscript with voltage bump executed")

In [ ]:
# Example 4: Fast pseudo bootscript (no actual boot)
dpm.pseudo_bs(
    ClassMask='FirstPass',
    boot=False,                 # Don't boot - just configure
    htdis=True,
    fast=True,                  # Fast mode
    fuse_read=True
)

print("✓ Fast pseudo bootscript (config only) executed")

### pseudo_bs - Available Class Masks

Common ClassMask options:
- `'FirstPass'` - First pass mask
- `'SecondPass'` - Second pass mask
- `'ThirdPass'` - Third pass mask
- Custom masks can be provided via `Custom=[]` parameter

### pseudo_bs - Use Cases

In [ ]:
# Use Case 1: Quick reconfiguration without reboot
# Useful for voltage sweeps where you need different configs
dpm.pseudo_bs(
    ClassMask='FirstPass',
    boot=False,             # No reboot
    htdis=True,
    fast=True
)

print("✓ Configuration changed without reboot")

In [ ]:
# Use Case 2: Test with specific core configuration
dpm.pseudo_bs(
    ClassMask='FirstPass',
    boot=True,
    htdis=True,
    dis_1CPM=True,
    dis_2CPM=True,          # Both CPM modes disabled
    fuse_read=True
)

print("✓ Pseudo bootscript with custom CPM configuration")

In [ ]:
# Use Case 3: PPVC fuse mode
dpm.pseudo_bs(
    ppvcfuse=True,          # Use PPVC fuses
    boot=True,
    htdis=True,
    fuse_read=True
)

print("✓ Pseudo bootscript with PPVC fuses")

---
## 3. BIOS Knobs Checking

Check and configure BIOS knobs.

### bsknobs - Bootscript Knobs

In [ ]:
# Check bootscript knobs (read-only)
dpm.bsknobs(readonly=True, skipinit=False)

print("✓ Bootscript knobs read and displayed")

In [ ]:
# Configure bootscript knobs (if readonly=False)
dpm.bsknobs(readonly=False, skipinit=False)

print("✓ Bootscript knobs configuration")

### biosknobs - BIOS Knobs Configuration

In [ ]:
# Read BIOS knobs (read-only mode)
dpm.biosknobs(knobs={}, readonly=True)

print("✓ BIOS knobs read")

In [ ]:
# Set specific BIOS knobs
my_knobs = {
    'knob1': 'value1',
    'knob2': 'value2'
}

dpm.biosknobs(knobs=my_knobs, readonly=False)

print("✓ BIOS knobs configured")

---
## 4. Fuse Management

Read and manage fuse settings.

### fuses - Read Fuse Settings

In [ ]:
# Read fuses from socket 0
fuse_data = dpm.fuses(
    rdFuses=True,       # Read fuses
    sktnum=[0],         # Socket 0
    printFuse=True      # Print to console
)

print("✓ Fuses read from socket 0")

In [ ]:
# Read fuses from multiple sockets
fuse_data = dpm.fuses(
    rdFuses=True,
    sktnum=[0, 1],      # Sockets 0 and 1
    printFuse=True
)

print("✓ Fuses read from sockets 0 and 1")

### fuseRAM - Fuse RAM Access

In [ ]:
# Access fuse RAM
dpm.fuseRAM(refresh=False)

print("✓ Fuse RAM accessed")

In [ ]:
# Refresh and access fuse RAM
dpm.fuseRAM(refresh=True)

print("✓ Fuse RAM refreshed and accessed")

---
## 5. Unit Information Functions

Get unit identification strings.

### visual_str - Get Visual ID

In [ ]:
# Get visual ID string from socket 0
visual_id = dpm.visual_str()

print(f"Visual ID: {visual_id}")

### qdf_str - Get QDF String

In [ ]:
# Get QDF string
qdf = dpm.qdf_str()

print(f"QDF: {qdf}")

### product_str - Get Product String

In [ ]:
# Get product string (GNR, CWF, DMR, etc.)
product = dpm.product_str()

print(f"Product: {product}")

### Combine Unit Information

In [ ]:
# Get all unit information at once
visual_id = dpm.visual_str()
qdf = dpm.qdf_str()
product = dpm.product_str()

print(f"Unit Information:")
print(f"  Product: {product}")
print(f"  Visual ID: {visual_id}")
print(f"  QDF: {qdf}")

---
## 6. Hardware Checks

Various hardware validation functions.

### u600w - 600W Unit Check

In [ ]:
# Check if unit is 600W
is_600w = dpm.u600w(check=True)

if is_600w:
    print("✓ Unit is 600W")
else:
    print("✓ Unit is not 600W")

### reset_600w - Reset 600W Unit

In [ ]:
# Reset 600W unit if applicable
dpm.reset_600w()

print("✓ 600W unit reset command sent")

### hwls_miscompare - HWLS Miscompare Check

In [ ]:
# Check for HWLS miscompare
dpm.hwls_miscompare()

print("✓ HWLS miscompare check performed")

---
## 7. Complete Workflow Example

Here's a complete workflow combining multiple dpmChecks functions:

In [ ]:
# Complete Test Workflow Example

# Step 1: Get unit information
print("=" * 60)
print("Step 1: Gathering unit information...")
print("=" * 60)

visual_id = dpm.visual_str()
qdf = dpm.qdf_str()
product = dpm.product_str()

print(f"Product: {product}")
print(f"Visual ID: {visual_id}")
print(f"QDF: {qdf}")

# Step 2: Check if 600W unit
print("\n" + "=" * 60)
print("Step 2: Checking unit type...")
print("=" * 60)

is_600w = dpm.u600w(check=True)
print(f"600W Unit: {is_600w}")

# Step 3: Initialize logger
print("\n" + "=" * 60)
print("Step 3: Initializing logger...")
print("=" * 60)

dpm.logger(
    visual=visual_id,
    qdf=qdf,
    TestName='CompleteWorkflow',
    Testnumber=1,
    dr_dump=True,
    Bucket='UNCORE'
)
print("✓ Logger initialized")

# Step 4: Read BIOS knobs
print("\n" + "=" * 60)
print("Step 4: Reading BIOS knobs...")
print("=" * 60)

dpm.bsknobs(readonly=True, skipinit=False)
print("✓ BIOS knobs read")

# Step 5: Read fuses
print("\n" + "=" * 60)
print("Step 5: Reading fuses...")
print("=" * 60)

fuse_data = dpm.fuses(rdFuses=True, sktnum=[0], printFuse=True)
print("✓ Fuses read")

# Step 6: Execute pseudo bootscript
print("\n" + "=" * 60)
print("Step 6: Executing pseudo bootscript...")
print("=" * 60)

dpm.pseudo_bs(
    ClassMask='FirstPass',
    boot=True,
    htdis=True,
    fuse_read=True
)
print("✓ Pseudo bootscript executed")

print("\n" + "=" * 60)
print("Workflow Complete!")
print("=" * 60)
print("\nNow run your test...")
print("Logger will capture any MCA errors")
print(f"Reports saved to: C:\\SystemDebug\\Logs\\MCA_Reports\\")

---
## 8. Advanced Examples

More advanced usage patterns for dpmChecks functions.

### Example: Voltage Sweep with Logging

In [ ]:
# Voltage sweep with MCA logging at each point

voltages = [0.90, 0.92, 0.94, 0.96, 0.98, 1.00]

for i, voltage in enumerate(voltages):
    print(f"\n{'='*60}")
    print(f"Testing voltage: {voltage}V (iteration {i+1}/{len(voltages)})")
    print(f"{'='*60}")

    # Initialize logger for this voltage point
    dpm.logger(
        visual=dpm.visual_str(),
        TestName=f'VoltageSweep_{voltage}V',
        Testnumber=i+1,
        dr_dump=True,
        Bucket='VOLTAGE_SWEEP'
    )

    # Configure system with voltage bump
    vbump_offset = voltage - 0.95  # Assuming 0.95V nominal

    dpm.pseudo_bs(
        ClassMask='FirstPass',
        boot=False,
        htdis=True,
        fast=True,
        vbump={
            'enabled': True,
            'type': ['cfc'],
            'offset': vbump_offset,
            'computes': ['compute0', 'compute1', 'compute2']
        }
    )

    print(f"✓ System configured for {voltage}V")
    print(f"  Run your test here...")
    print(f"  MCA errors will be logged to: C:\\SystemDebug\\Logs\\MCA_Reports\\")

print("\n✓ Voltage sweep configuration complete")

### Example: Multi-Socket Validation

In [ ]:
# Validate multiple sockets

sockets = [0, 1]

for socket in sockets:
    print(f"\n{'='*60}")
    print(f"Validating Socket {socket}")
    print(f"{'='*60}")

    # Read fuses for this socket
    fuse_data = dpm.fuses(
        rdFuses=True,
        sktnum=[socket],
        printFuse=True
    )

    # Get visual ID for this socket
    visual_id = dpm.visual_str()
    print(f"Socket {socket} Visual ID: {visual_id}")

    print(f"✓ Socket {socket} validated")

print("\n✓ Multi-socket validation complete")

---
## Summary

**dpmChecks Functions Summary:**

| Function | Purpose |
|----------|---------|
| `logger()` | MCA logging and error reporting |
| `pseudo_bs()` | Pseudo bootscript (no reboot) |
| `bsknobs()` | Bootscript knobs check |
| `biosknobs()` | BIOS knobs configuration |
| `fuses()` | Read fuse settings |
| `fuseRAM()` | Access fuse RAM |
| `visual_str()` | Get Visual ID |
| `qdf_str()` | Get QDF string |
| `product_str()` | Get product name |
| `u600w()` | Check 600W unit |
| `reset_600w()` | Reset 600W unit |
| `hwls_miscompare()` | Check HWLS miscompare |

**Common Workflows:**
1. **Test with Logging**: Initialize logger before test, captures MCA errors
2. **Configuration Changes**: Use pseudo_bs to change config without reboot
3. **Unit Validation**: Check unit info, fuses, BIOS knobs before testing
4. **Voltage Sweeps**: Combine logger + pseudo_bs for automated sweeps

**All functions work identically across GNR, CWF, and DMR products!**

---

## Summary & Best Practices

**Common Workflows:**
1. **Test with Logging**: Initialize logger before test, captures MCA errors
2. **Configuration Changes**: Use pseudo_bs to change config without reboot
3. **Unit Validation**: Check unit info, fuses, BIOS knobs before testing
4. **Voltage Sweeps**: Combine logger + pseudo_bs for automated sweeps

**All functions work identically across GNR, CWF, and DMR products!**

---

## Next Steps

1. Try the logger with your actual tests
2. Experiment with pseudo_bs for different configurations
3. Use unit information functions in your automation scripts
4. Review [THR_DEBUG_FRAMEWORK_EXAMPLES.ipynb](THR_DEBUG_FRAMEWORK_EXAMPLES.ipynb) for interface examples
5. Refer to [THR_DEBUG_FRAMEWORK_USER_MANUAL.md](THR_DEBUG_FRAMEWORK_USER_MANUAL.md) for complete documentation

**Important Locations:**
- MCA Reports: `C:\SystemDebug\Logs\MCA_Reports\`
- Framework Logs: `C:\SystemDebug\Logs\`
- Debug Logs: `C:\Temp\`

---

## 📧 Support

**Maintainer:** Gabriel Espinoza  
**Email:** gabriel.espinoza.ballestero@intel.com  
**Organization:** Intel Corporation - Test & Validation  
**Framework:** BASELINE Testing Framework v2.0  
**Release Date:** January 15, 2026  

**Complete Documentation:**
- User Manual: [THR_DEBUG_FRAMEWORK_USER_MANUAL.md](THR_DEBUG_FRAMEWORK_USER_MANUAL.md)
- File Naming: [THR_DEBUG_FRAMEWORK_FILE_NAMING_AND_IMPORTS.md](THR_DEBUG_FRAMEWORK_FILE_NAMING_AND_IMPORTS.md)
- DebugFramework Examples: [THR_DEBUG_FRAMEWORK_EXAMPLES.ipynb](THR_DEBUG_FRAMEWORK_EXAMPLES.ipynb)
- Main Index: [../README.md](../README.md)

**Repository:** `c:\Git\Automation\S2T\`  
**Documentation:** `c:\Git\Automation\S2T\DOCUMENTATION\`

---

**© 2026 Intel Corporation. Intel Confidential.**